In [56]:
import logging
import os
import pymongo
import cobrakbase
import time
%run ../../scripts/bios_utils.py
%run ../../annotation-server/annotation_api_neo4j.py

In [197]:
%run ../../annotation-server/annotation_api_neo4j.py
from neo4j import GraphDatabase
driver = GraphDatabase.driver("bolt://127.0.0.1:7687", auth=("neo4j", "123585"))
annotation_api = AnnotationApiNeo4j(driver)
#neo_annotation_api.init_constraints()

In [3]:
substem_ref = read_json('../../../data/tnseq/patric_subsystems/substem_ref.json')

In [4]:
%%time
for doc in substem_ref['response']['docs']:
    value = doc['subsystem_name']
    
    ss_node = None
    
    with driver.session() as session:
        ss_node = session.write_transaction(add_value_node, 'FunctionGroup', value)
    
    if 'role_name' in doc:
        for role in doc['role_name']:
            role = role.strip()
            if len(role) > 0:
                with driver.session() as session:
                    function_node = session.write_transaction(add_value_node, 'Function', role)
                    session.write_transaction(add_relationship, 
                                    ss_node, function_node, 'has_function')

NameError: name 'driver' is not defined

In [9]:
modelseed_local = cobrakbase.modelseed.from_local('../../../ModelSEEDDatabase')

load: ../../../ModelSEEDDatabase/Biochemistry/reactions.tsv
load: ../../../ModelSEEDDatabase/Biochemistry/compounds.tsv
load: ../../../ModelSEEDDatabase/Biochemistry/Structures/Unique_ModelSEED_Structures.txt
load: ../../../ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt
load: ../../../ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt
load: ../../../ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt


In [17]:
node = annotation_api.get_node('RefSeqGenome', 'GCF_000007025.1')
print(node)

<Node id=57 labels={'RefSeqGenome'} properties={'genetic_code': 11, 'notes': '', 'release': '1', 'created_at': 1563767041245, 'scientific_name': 'Rickettsia conorii str. Malish 7', 'source': 'refseq', 'taxonomy': 'cellular organisms; Bacteria; Proteobacteria; Alphaproteobacteria; Rickettsiales; Rickettsiaceae; Rickettsieae; Rickettsia; spotted fever group; Rickettsia conorii', 'type': 'Representative', 'external_source_origination_date': '16-NOV-2016', 'count_features': 1532, 'workspace_id': 'jplfaria:narrative_1524466549180', 'dna_size': 1268755, 'original_source_file_name': 'GCF_000007025.1_ASM702v1_genomic.gbff', 'reference_annotation': 0, 'updated_at': 1563767041245, 'num_contigs': 1, 'assembly_ref': '19217/46804/1', 'domain': 'Bacteria', 'source_id': 'NC_003103', 'id': 'GCF_000007025.1', 'taxon_ref': '12570/666753/1', 'genbank_handle_ref': 'KBH_235629', 'gc_content': 0.3243880812292365, 'key': 'GCF_000007025.1'}>


In [21]:
rxn.data

{'id': 'rxn00001',
 'abbreviation': 'R00004',
 'name': 'diphosphate phosphohydrolase',
 'code': '(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0]',
 'stoichiometry': '-1:cpd00001:0:0:"H2O";-1:cpd00012:0:0:"PPi";2:cpd00009:0:0:"Phosphate";1:cpd00067:0:0:"H+"',
 'is_transport': 0,
 'equation': '(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0] + (1) cpd00067[0]',
 'definition': '(1) H2O[0] + (1) PPi[0] <=> (2) Phosphate[0] + (1) H+[0]',
 'reversibility': '>',
 'direction': '=',
 'abstract_reaction': nan,
 'pathways': 'MetaCyc: Degradation (Degradation/Utilization/Assimilation); Glyphosate-Degradation (glyphosate degradation); Noncarbon-Nutrients (Inorganic Nutrient Metabolism); PWY-7805 ((aminomethyl)phosphonate degradation); PWY-7807 (glyphosate degradation III); Phosphorus-Compounds (Phosphorus Compound Metabolism)',
 'aliases': 'AraCyc: INORGPYROPHOSPHAT-RXN|BiGG: IPP1; PPA; PPA_1; PPAm|BrachyCyc: INORGPYROPHOSPHAT-RXN|KEGG: R00004|MetaCyc: INORGPYROPHOSPHAT-RXN|Name: Diphosp

In [54]:
props_str = to_str_dict(props)
props_str

'{name:"a \\"banana\\"", key:"banana"}'

In [55]:
props = {
    'name' : 'a "banana"'
}
add_node(annotation_api, 'Fruit', 'banana', props)

<Node id=97399 labels={'Fruit'} properties={'name': 'a "banana"', 'created_at': 1568612758221, 'updated_at': 1568612758221, 'key': 'banana'}>

In [24]:
props

{'id': 'rxn10881',
 'name': 'streptomycin 3"-adenylyltransferase',
 'equation': '(1) cpd00002[0] + (1) cpd00067[0] + (1) cpd00328[0] <= (1) cpd00012[0] + (1) cpd16061[0]',
 'definition': '(1) ATP[0] + (1) H+[0] + (1) Streptomycin[0] <= (1) PPi[0] + (1) 3"-Adenylylstreptomycin[0]',
 'key': 'rxn10881'}

abstract_reaction
pathways
ec_numbers
linked_reaction


# Load ModelSEED Reactions

In [63]:
for seed_id in modelseed_local.reactions:
    rxn = modelseed_local.get_seed_reaction(seed_id)
    rxn_node = annotation_api.get_node('ModelSeedReaction', seed_id)
    if rxn_node == None:
        delete = set()
        for k in rxn.data:
            if pd.isna(rxn.data[k]):
                delete.add(k)
        for k in delete:
            del rxn.data[k]
        annotation_api.add_node('ModelSeedReaction', seed_id, rxn.data)

# Load KEGG Orthologs

In [68]:
df_kegg_ko = pd.read_csv('../../data/ko_kegg.tsv', sep='\t')
for row_id, d in df_kegg_ko.iterrows():
    kegg_ko = d['kegg_ko']
    kegg_rxns = d['kegg_rxns']
    
    ko_node = annotation_api.get_node('KeggOrthology', kegg_ko)
    if ko_node == None:
        annotation_api.add_node('KeggOrthology', kegg_ko, {})
        ko_node = annotation_api.get_node('KeggOrthology', kegg_ko)
        
    if not pd.isna(kegg_rxns):
        for rxn_id in kegg_rxns.split(';'):
            rxn_node = annotation_api.get_node('LigandReaction', seed_id)
            if rxn_node == None:
                annotation_api.add_node('LigandReaction', rxn_id, {})
                rxn_node = annotation_api.get_node('LigandReaction', rxn_id)
            annotation_api.link_nodes(ko_node, rxn_node, 'has_reaction')

# Load KEGG Orthologs to SEED

In [72]:
for seed_id in modelseed_local.reactions:
    rxn = modelseed_local.get_seed_reaction(seed_id)
    seed_rxn_node = annotation_api.get_node('ModelSeedReaction', seed_id)
    if seed_id in modelseed_local.reaction_aliases and 'KEGG' in modelseed_local.reaction_aliases[seed_id]:
        for kegg_id in modelseed_local.reaction_aliases[seed_id]['KEGG']:
            kegg_rxn_node = annotation_api.get_node('LigandReaction', kegg_id)
            if not kegg_rxn_node == None and not seed_rxn_node == None:
                annotation_api.link_nodes(seed_rxn_node, kegg_rxn_node, 'has_crossreference_to')

# Load Templates

In [76]:
kbase = cobrakbase.KBaseAPI(token='YAFOCRSMRNDXZ7KMW7GCK5AC3SBNTEFD')
templates = {}
templates['GramNegModelTemplateV2'] = kbase.get_object('GramNegModelTemplateV2', 'NewKBaseModelTemplates')
templates['GramNegModelTemplate'] = kbase.get_object('GramNegModelTemplate', 'NewKBaseModelTemplates')
#template_doc = annotation_api.add_template('GramNegModelTemplateV2', templates['GramNegModelTemplateV2'])
#template_doc = annotation_api.add_template('GramNegModelTemplate', templates['GramNegModelTemplate'])

In [143]:
template_node = annotation_api.get_node('TemplateSet', template_id)
if template_node == None:
    annotation_api.add_node('TemplateSet', template_id, {})
    template_node = annotation_api.get_node('TemplateSet', template_id)
template_node

<Node id=172061 labels={'TemplateSet'} properties={'updated_at': 1568620759396, 'created_at': 1568620759396, 'key': 'GramNegModelTemplate'}>

In [214]:

add_template_reaction_annotation(annotation_api, template_node, rxn_node, rxn_compartments, or_rule, props)

<Node id=185606 labels={'TemplateReactionAnnotation'} properties={'reaction': 'rxn00879', 'updated_at': 1568635675119, 'forward_penalty': 0, 'compartment': 'c', 'base_cost': 2, 'reverse_penalty': 0, 'created_at': 1568635675119, 'GapfillDirection': '=', 'type': 'conditional', 'key': 'rxn09225_c', 'direction': '='}>

In [170]:

def get_relationship(tx, label, key, relationship_type):
    result = tx.run("MATCH (n1:ModelSeedReaction {key:$key})-[r]-(n2) RETURN r", key = key)
    return list(result.records())

In [172]:
#rxn_doc = annotation_api.get_node('ModelSeedReaction', seed_id)
def get_r_nodes(annotation_api, label, key, relationship_type = None):
    result = None
    with annotation_api.driver.session() as session:
        query = session.read_transaction(get_relationship, label, key, relationship_type)
        if len(query) > 0:
            print(query[0].data())
            #result = query[0].data()['n']
    return result
get_r_nodes(annotation_api, 'ModelSeedReaction', 'rxn00004')

{'r': <Relationship id=99688 nodes=(<Node id=87034 labels=set() properties={}>, <Node id=133792 labels=set() properties={}>) type='has_crossreference_to' properties={}>}


In [149]:
for r in template_node.graph.relationships:
    print(r)

In [152]:
def get_ko_by_seed_id(annotation_api, seed_id):
    rxn_doc = annotation_api.get_node('ModelSeedReaction', seed_id)
    print(rxn_doc)
    if not rxn_doc == None:
        for r in rxn_doc.graph.relationships:
            print(r)
        if 'kegg_kos' in rxn_doc:
            return set(rxn_doc['kegg_kos'])
        else:
            return set()
    return None
get_ko_by_seed_id(annotation_api, 'rxn00004')

<Node id=87034 labels={'ModelSeedReaction'} properties={'updated_at': 1568609814074, 'equation': '(1) cpd02570[0] <=> (2) cpd00020[0]', 'name': '4-hydroxy-4-methyl-2-oxoglutarate pyruvate-lyase (pyruvate-forming)', 'created_at': 1568609814074, 'definition': '(1) Parapyruvate[0] <=> (2) Pyruvate[0]', 'id': 'rxn00004', 'key': 'rxn00004'}>


set()

In [ ]:
def get_reaction_annotation_data(rxn_id):
    kos = annotation_api.get_ko_by_seed_id(rxn_id)
    function_data = {}
    for ko in kos:
        #print(ko)
        functions, functions_data, metadata = get_functional_roles2(ko, annotation_api)
        print(ko, metadata)
        for f in functions:
            function = functions_data[f]['value']
            if not function in function_data:
                function_data[function] = {
                    'id' : f,
                    'hits' : []
                }
            function_data[function]['hits'].append({
                'score' : len(functions[f]),
                'source' : ['KEGG', ko]
            })
    for template_doc in annotation_api.collection_templates.find():
        if rxn_id in template_doc['reactions']:
            template_id = template_doc['_id']
            template_reaction_doc = template_doc['reactions'][rxn_id]
            for and_rule in template_reaction_doc['functions']:
                for function_id in and_rule:
                    function_doc = annotation_api.collection_functions.find_one({'_id' : function_id})
                    function = function_doc['value']
                    if not function in function_data:
                        function_data[function] = {
                            'id' : function_id,
                            'hits' : []
                        }
                    function_data[function]['hits'].append({
                        'score' : 0,
                        'source' : ['template', template_id]
                    })
                    #print(template_id, function_doc['value'])
        
    for function in function_data:
        function_id = function_data[function]['id']
        #print(function_id)
        function_doc = annotation_api.collection_functions.find_one({'_id' : function_id})
        function_metadata = annotation_api.collection_functions_data.find_one({'_id' : function_id})
        
        subsystem_tags = {}
        if 'subsystem' in function_doc:
            for subsystem_tag in function_doc['subsystem']:
                subsystem_tags[subsystem_tag] = [function_doc['subsystem'][subsystem_tag]]
                if 'subsystem_class' in function_doc and subsystem_tag in function_doc['subsystem_class']:
                    subsystem_tags[subsystem_tag].append(function_doc['subsystem_class'][subsystem_tag])
        #print(subsystem_tags)
        
        
        source_tags = {}
        if not function_metadata == None:
            sources = function_metadata['sources']
            for s in sources:
                source_tags[s] = [len(sources[s]['genomes']), len(sources[s]['genes']), sources[s]['genes'][:10]]
            #print(source_tags)
        function_data[function]['sources'] = source_tags
        function_data[function]['subsystems'] = subsystem_tags
    return function_data